In [ ]:
# https://www.analyticsvidhya.com/blog/2018/11/introduction-text-summarization-textrank-python/
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') # one time execution
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2020-11-11 22:54:39--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-11-11 22:54:40--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-11-11 22:54:40--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [ ]:
len(word_embeddings)


400000

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [79]:
import pickle
filepath= '/content/drive/My Drive/Colab Notebooks/data/sample_cnn_dataset.pkl'
stories = pickle.load(open(filepath, 'rb'))
print(len(stories))
sentences = stories[1]['story']

# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in sentences]
clean_sentences

450


['usain bolt rounded world championships sunday claiming third gold moscow anchored jamaica victory mens relay',
 'fastest man world charged clear united states rival justin gatlin jamaican quartet nesta carter kemar baileycole nickel ashmeade bolt seconds',
 'us finished second seconds canada taking bronze britain disqualified faulty handover',
 'bolt collected eight gold medals world championships equaling record held american trio carl lewis michael johnson allyson felix mention small matter six olympic titles',
 'relay triumph followed individual successes meters russian capital',
 'im proud ill continue work dominate long possible bolt said previously expressed intention carry rio olympics',
 'victory never seriously doubt got baton safely hand ashmeade gatlin united states third leg runner rakieem salaam problems',
 'gatlin strayed lane struggled get full control baton never able get terms bolt',
 'earlier jamaicas women underlined dominance sprint events winning relay gold ancho

In [80]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [81]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)
# sentence_vectors

In [82]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [83]:
from sklearn.metrics.pairwise import cosine_similarity
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [84]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [85]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [86]:
# Extract top 3 sentences as the summary
for i in range(3):
  print(ranked_sentences[i][1])

in other final action on the last day of the championships frances teddy tamgho became the third man to leap over in the triple jump exceeding the mark by four centimeters to take gold
defending champions the united states were initially back in the bronze medal position after losing time on the second handover between alexandria anderson and english gardner but promoted to silver when france were subsequently disqualified for an illegal handover
the bolt has now collected eight gold medals at world championships equaling the record held by american trio carl lewis michael johnson and allyson felix not to mention the small matter of six olympic titles


In [78]:
stories[1]['highlights']

['usain bolt wins third gold of world championship',
 'anchors jamaica to relay victory',
 'eighth gold at the championships for bolt',
 'jamaica double up in womens relay']